<font size="6"> Table of Contents</font>
<a id='destination_cell_10'></a> <br>
[Logistic_Regression_1_selftext](#destination_cell_3) <br>
[Logistic_Regression_2_title](#destination_cell_4)<br>
[Regression Tree (CART)](#destination_cell_6) <br>
[Random Forest](#destination_cell_17) <br>
[Knn](#destination_cell_18) <br>
[AdaBoost](#destination_cell_16) <br>
[Stacking](#destination_cell_5) <br>

In [1]:
import pandas as pd
import numpy as np
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import re

from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import resample


from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize

from collections import Counter

In [2]:
%cd '/Users/nicholassanso/Desktop/GA/projects/project-3'

/Users/nicholassanso/Desktop/GA/projects/project-3


In [3]:
#load the CSV file
df_combined = pd.read_csv('./post_grad_p3/df_combined.csv')


<font size="8">Logistic_Regression_1_selftext</font>
<a id='destination_cell_3'></a>

[Return to Table of Contents](#destination_cell_10)

In [4]:
X = df_combined['selftext_cleaned_1_lemmatized']
y = df_combined['subreddit']

#train test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=42)
X_pred, X_test, y_pred, y_test = train_test_split(X_val, y_val, test_size=0.25, random_state=42)

In [5]:
tvec = TfidfVectorizer(stop_words='english')
logreg = LogisticRegression(solver = 'liblinear', random_state= 42, max_iter=1000)

#create a pipeline
pipe_selftext = Pipeline([
    ('tvec', tvec),
    ('logreg', logreg)
])

# define the parameter grid
param_grid = {
    'tvec__ngram_range': [(1, 1), (1, 2)],
    'logreg__C': [0.1, 1, 10],
    'logreg__penalty': ['l1', 'l2']
}

#create a GridSearchCV instance
grid_search_logreg_1 = GridSearchCV(pipe_selftext, param_grid)

#fit the GridSearchCV instance on the data 
grid_search_logreg_1.fit(X_train, y_train)

print(grid_search_logreg_1.best_params_)

{'logreg__C': 10, 'logreg__penalty': 'l2', 'tvec__ngram_range': (1, 2)}


In [6]:
#fit on the training data
grid_search_logreg_1.fit(X_train, y_train)

#the accuracy of the training data
train_acc = grid_search_logreg_1.score(X_train, y_train)
print(train_acc)

#the accuracy of the validation data
val_acc = grid_search_logreg_1.score(X_val, y_val)
print(val_acc)

0.9955555555555555
0.7722222222222223


In [7]:
scores = cross_val_score(grid_search_logreg_1, X_train, y_train, cv=5)
print(scores)
print(scores.mean())
print(scores.std())

[0.74814815 0.78148148 0.74814815 0.75925926 0.76851852]
0.7611111111111112
0.012722640027642711


In [8]:
predictions_selftext = grid_search_logreg_1.predict(X_pred)
predictions_selftext.shape

(1350,)

In [9]:
X_test = X_test.values

In [10]:
predictions_selftext_X_test = grid_search_logreg_1.predict(X_test)
predictions_selftext_X_test.shape

(450,)

<font size="8">Logistic_Regression_2_title</font>
<a id='destination_cell_4'></a>

[Return to Table of Contents](#destination_cell_10)

In [11]:
X = df_combined['title_cleaned_1_lemmatized']
y = df_combined['subreddit']

#train test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=42)
X_pred, X_test, y_pred, y_test = train_test_split(X_val, y_val, test_size=0.25, random_state=42)

In [12]:
#instantiate the TfidVectorizer and LogisticRegression
tvec_2 = TfidfVectorizer(stop_words='english')
logreg_2 = LogisticRegression(solver = 'liblinear', random_state= 42, max_iter=1000)

#pipeline
pipe_title = Pipeline([
    ('tvec_2', tvec_2),
    ('logreg_2', logreg_2)
])

#define the parameter grid
param_grid = {
    'tvec_2__ngram_range': [(1, 1), (1, 2)],
    'logreg_2__C': [0.1, 1],
    'logreg_2__penalty': ['l1', 'l2']
}

#instantiate a GridSearchCV
grid_search_logreg_2 = GridSearchCV(pipe_title, param_grid)

#fit the GridSearch"
grid_search_logreg_2.fit(X_train, y_train)

print(grid_search_logreg_2.best_params_);

{'logreg_2__C': 1, 'logreg_2__penalty': 'l2', 'tvec_2__ngram_range': (1, 1)}


In [13]:
print(grid_search_logreg_2.score(X_train, y_train))
print(grid_search_logreg_2.score(X_val, y_val))

0.882962962962963
0.7433333333333333


In [14]:
scores = cross_val_score(grid_search_logreg_2, X_train, y_train, cv=5)
print(scores)
print(scores.mean())
print(scores.std())

[0.72222222 0.73518519 0.70740741 0.75       0.73333333]
0.7296296296296296
0.014200214453837649


In [15]:
#the accuracy on the training data
train_acc = grid_search_logreg_2.score(X_train, y_train)

#the accuracy on the validation data
val_acc = grid_search_logreg_2.score(X_val, y_val)

print(train_acc)
print(val_acc)

0.882962962962963
0.7433333333333333


In [16]:
predictions_title = grid_search_logreg_2.predict(X_pred)
predictions_title.shape

(1350,)

In [17]:
X_test = X_test.values

In [18]:
predictions_title_X_test = grid_search_logreg_2.predict(X_test)
predictions_title_X_test.shape

(450,)

<font size="8">Train, Validation Split and Standard Scaler </font>
<a id='destination_cell_6'></a>

[Return to Table of Contents](#destination_cell_10)

In [19]:
X=df_combined[['ups','num_comments','total_awards_received']]
y=df_combined['subreddit']

#train test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=42)
X_pred, X_test, y_pred, y_test = train_test_split(X_val, y_val, test_size=0.25, random_state=42)

In [20]:
sc = StandardScaler()

#learn the mean, stdev of each feature in the fit part, then applies the transformation
X_train_sc = sc.fit_transform(X_train)

In [21]:
#we don't fit on test set, we only transform it with mean, stdev learned from training set
X_pred_sc = sc.transform(X_pred)

<font size="8">Regression Tree (CART) </font>
<a id='destination_cell_6'></a>

[Return to Table of Contents](#destination_cell_10)

In [22]:
#define the parameter grid
param_grid = {
    'max_depth': [5, 10, 15, 20],
    'min_samples_leaf': [5, 8, 10],
}

#instantiate cart estimator 
cart = DecisionTreeClassifier(random_state=42)

#instantiate GridSearchCV 
grid_search_cart = GridSearchCV(cart, param_grid)

#fit the GridSearchCV to the data
grid_search_cart.fit(X_train_sc, y_train)

print(grid_search_cart.best_params_);

{'max_depth': 5, 'min_samples_leaf': 5}


In [23]:
#compute the accuracy on the training data
train_cart_accuracy = grid_search_cart.score(X_train_sc, y_train)

#compute the accuracy on the validation data
validation_cart_accuracy = grid_search_cart.score(X_pred_sc, y_pred)

print(train_cart_accuracy)
print(validation_cart_accuracy)

0.6811111111111111
0.642962962962963


In [24]:
scores = cross_val_score(grid_search_cart, X_train_sc, y_train, cv=5)
print(scores)
print(scores.mean())
print(scores.std())

[0.65       0.67592593 0.66666667 0.66481481 0.65555556]
0.6625925925925926
0.009026709338484394


In [25]:
predictions_cart = grid_search_cart.predict(X_pred_sc)
predictions_cart.shape

(1350,)

In [26]:
X_test = X_test.values

In [27]:
predictions_cart_X_test = grid_search_cart.predict(X_test)
predictions_cart_X_test.shape

(450,)

<font size="8">Random Forest </font>
<a id='destination_cell_17'></a>

[Return to Table of Contents](#destination_cell_10)

In [28]:
#define the parameter grid
param_grid = {
    'max_depth': [5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 5, 10],
    'ccp_alpha': [0.0, 0.1, 0.2, 0.3]
}

#create a RandomForestClassifier instance
rf = RandomForestClassifier(random_state=42)

#instantiate a GridSearchCV
grid_search_rf = GridSearchCV(rf, param_grid)

#fit the GridSearchCV 
grid_search_rf.fit(X_train_sc, y_train)

GridSearchCV(estimator=RandomForestClassifier(random_state=42),
             param_grid={'ccp_alpha': [0.0, 0.1, 0.2, 0.3],
                         'max_depth': [5, 10, 15, 20],
                         'min_samples_leaf': [1, 2, 5, 10]})

In [29]:
#compute the accuracy on the training data
train_rf_accuracy = grid_search_rf.score(X_train_sc, y_train)

#compute the accuracy on the validation data
validation_rf_accuracy = grid_search_rf.score(X_pred_sc, y_pred)

print(train_rf_accuracy)
print(validation_rf_accuracy)

0.727037037037037
0.6451851851851852


In [30]:
cross_val_scores = cross_val_score(grid_search_rf, X_train_sc, y_train, cv=5)
print(cross_val_scores)
print(cross_val_scores.mean())
print(cross_val_scores.std())

[0.65       0.6537037  0.67592593 0.67592593 0.66111111]
0.6633333333333333
0.01088662107903635


In [31]:
predictions_rf = grid_search_rf.predict(X_pred_sc)
predictions_rf.shape

(1350,)

In [32]:
predictions_rf_X_test = grid_search_rf.predict(X_test)
predictions_rf_X_test.shape

(450,)

<font size="8">k_nearest_neighbors </font>
<a id='destination_cell_18'></a>

[Return to Table of Contents](#destination_cell_10)

In [33]:
#define the parameter grid
param_grid = {
    'n_neighbors': range(10, 20)
}

#instantiate knn 
knn = KNeighborsClassifier()

#create a GridSearchCV instance
grid_search_knn = GridSearchCV(knn, param_grid)

#fit the GridSearchCV instance to the data
grid_search_knn.fit(X_train_sc, y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(10, 20)})

In [34]:
#compute the accuracy on the training data
train_knn_acc = grid_search_knn.score(X_train_sc, y_train)

#compute the accuracy on the validation data
val_knn_acc = grid_search_knn.score(X_pred_sc, y_pred)

print(train_knn_acc)
print(val_knn_acc)

0.6892592592592592
0.6311111111111111


In [35]:
scores = cross_val_score(grid_search_knn, X_train_sc, y_train, cv=5)
print(scores)
print(scores.mean())
print(scores.std())

[0.63518519 0.64444444 0.66111111 0.67407407 0.65925926]
0.6548148148148147
0.013588100983588234


In [36]:
predictions_knn = grid_search_knn.predict(X_pred_sc)
predictions_knn.shape

(1350,)

In [37]:
predictions_knn_X_test = grid_search_knn.predict(X_test)
predictions_knn_X_test.shape

(450,)

<font size="8">Adaboost </font>
<a id='destination_cell_16'></a>

[Return to Table of Contents](#destination_cell_10)

In [38]:
X=df_combined[[
       'char_over_sent_title', 'char_over_word_title', 'word_over_sent_title',
       'char_over_sent_selftext', 'char_over_word_selftext',
       'word_over_sent_selftext']]
y=df_combined['subreddit']

#train test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=42)
X_pred, X_test, y_pred, y_test = train_test_split(X_val, y_val, test_size=0.25, random_state=42)

In [39]:
#define the parameter grid
param_grid = {
    'learning_rate': [0.01, 0.1, 1.0]
}
 
#instantiate adaboost
ada = AdaBoostClassifier(random_state=42)

#create a GridSearchCV 
grid_search_ada = GridSearchCV(ada, param_grid)

#fit the GridSearchCV 
grid_search_ada.fit(X_train, y_train)

GridSearchCV(estimator=AdaBoostClassifier(random_state=42),
             param_grid={'learning_rate': [0.01, 0.1, 1.0]})

In [40]:
#compute the accuracy on the training data
train_ada_acc = grid_search_ada.score(X_train, y_train)
print(train_ada_acc)

0.6985185185185185


In [41]:
# compute the accuracy on the validation data
val_ada_acc = grid_search_ada.score(X_pred, y_pred)
print(val_ada_acc)

0.6962962962962963


In [42]:
scores = cross_val_score(grid_search_ada, X_train_sc, y_train, cv=5)
print(scores)
print(scores.mean())
print(scores.std())

[0.64259259 0.6462963  0.67777778 0.6537037  0.65      ]
0.654074074074074
0.0124170774920298


In [43]:
predictions_ada = grid_search_ada.predict(X_pred)
predictions_ada.shape

(1350,)

In [44]:
X_test = X_test.values

In [45]:
predictions_ada_X_test = grid_search_ada.predict(X_test)
predictions_ada_X_test.shape

/Users/nicholassanso/anaconda3/envs/dsi/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(


(450,)

<font size="8">Stacking </font>
<a id='destination_cell_5'></a>

[Return to Table of Contents](#destination_cell_10)

In [46]:
#initialize
le = LabelEncoder()

#fit the LabelEncoder 
y_pred_encoded = le.fit_transform(y_pred)

In [47]:
#transform your test labels 
y_test_encoded = le.transform(y_test)

In [68]:
#encode the predictions
predictions_ada_encoded = le.transform(predictions_ada)
predictions_knn_encoded = le.transform(predictions_knn)
predictions_rf_encoded = le.transform(predictions_rf)
predictions_cart_encoded = le.transform(predictions_cart)
predictions_title_encoded = le.transform(predictions_title)
predictions_selftext_encoded = le.transform(predictions_selftext)

predictions_ada_X_test_encoded = le.transform(predictions_ada_X_test)
predictions_knn_X_test_encoded = le.transform(predictions_knn_X_test)
predictions_rf_X_test_encoded = le.transform(predictions_rf_X_test)
predictions_cart_X_test_encoded = le.transform(predictions_cart_X_test)
predictions_title_X_test_encoded = le.transform(predictions_title_X_test)
predictions_selftext_X_test_encoded = le.transform(predictions_selftext_X_test)

In [69]:
#stack all encoded predictions together
X_train_stacked= np.column_stack((predictions_ada_encoded, predictions_knn_encoded, predictions_rf_encoded, 
                                         predictions_cart_encoded, predictions_title_encoded, 
                                         predictions_selftext_encoded))
#stack all encoded predictions together
X_test_stacked= np.column_stack((predictions_ada_X_test_encoded, predictions_knn_X_test_encoded,
                                 predictions_rf_X_test_encoded, predictions_cart_X_test_encoded, 
                                 predictions_title_X_test_encoded, predictions_selftext_X_test_encoded))

### Stacked logistic Regression

In [70]:
# instantiate
lr_stacked = LogisticRegression(C=0.1, random_state=42)
lr_stacked.fit(X_train_stacked, y_pred_encoded)

LogisticRegression(C=0.1, random_state=42)

The large dropoff in accuracy between the training dataset and the test data indicates that the model is deeply overfit.

In [53]:
print(lr_stacked.score(X_train_stacked,y_pred_encoded))

0.7762962962962963


In [71]:
print(lr_stacked.score(X_test_stacked, y_test_encoded))

0.7911111111111111


In [72]:
scores = cross_val_score(lr_stacked, X_train_stacked, y_pred_encoded, cv=5)
print(scores)
print(scores.mean())
print(scores.std())

[0.81111111 0.75925926 0.77407407 0.75555556 0.78518519]
0.7770370370370371
0.020068469902435656


In [84]:
scores = cross_val_score(lr_stacked, X_test_stacked, y_test_encoded, cv=5)
print(scores)
print(scores.mean())
print(scores.std())

[0.77777778 0.87777778 0.77777778 0.84444444 0.72222222]
0.8
0.05488484015657097


### Stacked AdaBoost

In [87]:
#define the parameter grid
param_grid = {
    'learning_rate': [0.01, 0.1, 1.0, 2.0]
}
 
#instantiate adaboost
ada = AdaBoostClassifier(random_state=42)

#create a GridSearchCV 
grid_search_ada = GridSearchCV(ada, param_grid)

#fit the GridSearchCV 
grid_search_ada.fit(X_train_stacked, y_pred_encoded)

print(grid_search_ada.best_params_);

{'learning_rate': 0.1}


In [75]:
print(grid_search_ada.score(X_train_stacked,y_pred_encoded))

0.7748148148148148


In [77]:
print(grid_search_ada.score(X_test_stacked, y_test_encoded))

0.8022222222222222


In [82]:
scores = cross_val_score(grid_search_ada, X_train_stacked, y_pred_encoded, cv=5)
print(scores)
print(scores.mean())
print(scores.std())

[0.80740741 0.75555556 0.77407407 0.75555556 0.78518519]
0.7755555555555556
0.019542082901145095


In [83]:
scores = cross_val_score(grid_search_ada, X_test_stacked, y_test_encoded, cv=5)
print(scores)
print(scores.mean())
print(scores.std())

[0.77777778 0.84444444 0.76666667 0.82222222 0.72222222]
0.7866666666666667
0.04297573245736381
